In [1]:
import numpy as np
import pandas as pd

In [2]:
# 读入基础数据
data = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\pre\\train.csv')
data['date'] = data['clickTime'].apply(lambda x: int(x/10000))
data = data[['label', 'date']]

In [3]:
# 读入特征
def readFeature(features):
    for feature in features:
        global data #声明data是global而不是局部变量
        dataFeature = pd.read_csv('C:\\Users\\work\\Desktop\\tencent\\feature\\train\\' + feature + '.csv', names=[feature])
        data = pd.concat([data, dataFeature], axis=1)
readFeature(['connectionType','telecomsOperator', 'lastNCVR_appID', 'lastNCVR_advertiserID'])

In [5]:
data.head()

,label,date,connectionType,telecomsOperator,lastNCVR_appID,lastNCVR_advertiserID
0,0,17,1,1,0.024873,0.024873
1,0,17,1,2,0.024873,0.024873
2,0,17,4,1,0.024873,0.024873
3,0,17,1,1,0.024873,0.024873
4,0,17,1,1,0.024873,0.024873


In [8]:
# 特征哑变量化与离散化
import sklearn.preprocessing as preprocessing
## 离散变量哑变量化
def featureDummy(features):
    for feature in features:
        global data #声明data是global而不是局部变量
        dummies = pd.get_dummies(data[feature], prefix=feature)
        data = pd.concat([data, dummies], axis=1)
        data.drop([feature], axis=1, inplace=True)
## 连续变量标准化
def featureScale(features):
    for feature in features:
        global data #声明data是global而不是局部变量
        scaler = preprocessing.StandardScaler()
        scaleParam = scaler.fit(data[feature])
        data[feature] = scaler.fit_transform(data[feature], scaleParam)

# 哑变量化与离散化
featureDummy(['connectionType', 'telecomsOperator'])
featureScale(['lastNCVR_appID', 'lastNCVR_advertiserID'])

C:\Users\work\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\work\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\work\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single featur

In [9]:
data.head()

,label,date,lastNCVR_appID,lastNCVR_advertiserID,connectionType_0,connectionType_1,connectionType_2,connectionType_3,connectionType_4,connectionType_0,connectionType_1,connectionType_2,connectionType_3,connectionType_4,telecomsOperator_0,telecomsOperator_1,telecomsOperator_2,telecomsOperator_3
0,0,17,-0.029142,-0.033222,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0,17,-0.029142,-0.033222,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,17,-0.029142,-0.033222,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0,17,-0.029142,-0.033222,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0,17,-0.029142,-0.033222,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [9]:
# 提取特征
# all_data = data.filter(regex='label|connectionType_*|telecomsOperato_*|appPlatform_*|lastNCVR_appID_+|lastNCVR_advertiserID_+')

In [11]:
# 平衡正反例
def balance(train_pro):
    ## 从正例中随机取出比例为a的数据，划分
    train_size_p = int(data[data['label']==1]['label'].count()*train_pro)
    train_size_n = int(train_size_p * 39.02) #原始样本中负例是正例的这个倍数
    sampler1 = np.random.permutation(data[data['label']==1]['label'].count())
    index1 = data[data['label']==1].index[sampler1] #重排序后的索引
    train1 = data[data['label']==1].ix[index1[:train_size_p]]
    test1 = data[data['label']==1].ix[index1[train_size_p:]]
    ## 从负例中随机取出相同数量的数据，划分
    sampler0 = np.random.permutation(data[data['label']==0]['label'].count())
    index0 = data[data['label']==0].index[sampler0] #重排序后的索引
    train0 = data[data['label']==0].ix[index0[:train_size_n]]
    test0 = data[data['label']==0].ix[index0[train_size_n:]]
    ## 构建训练集和测试集
    data_train = pd.concat([train1, train0])
    data_test = pd.concat([test1, test0])
    return data_train.as_matrix()[:,1:], data_test.as_matrix()[:,1:], data_train.as_matrix()[:,0],  data_test.as_matrix()[:,0]
trainX, testX, trainY, testY = balance(0.8)

In [12]:
# 划分训练集和测试集
from sklearn import cross_validation
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
# X = all_data.as_matrix()[:,1:]
# y = all_data.as_matrix()[:,0]
# trainX, testX, trainY, testY = cross_validation.train_test_split(X, y, test_size=0.2, random_state=174)
# y
# 训练
clf = linear_model.LogisticRegressionCV(Cs=10, penalty='l2', tol=1e-4, n_jobs=-1, cv=5)
# clf = RandomForestClassifier()
clf.fit(trainX, trainY)
print clf

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)


In [13]:
# 评估模型结果
import scipy as sp
def logloss(act, pred):
  epsilon = 1e-15
  pred = sp.maximum(epsilon, pred)
  pred = sp.minimum(1-epsilon, pred)
  ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
  ll = ll * -1.0/len(act)
  return ll
from sklearn import metrics
pred = clf.predict(testX)
print 'P&R'
print metrics.classification_report(testY, pred)
print 'loss:'
act = clf.predict_proba(testX)[:, 1]
# scaleParam = scaler.fit(act)
# act = scaler.fit_transform(act, scaleParam)
print logloss(act, pred)

P&R


C:\Users\work\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

        0.0       0.98      1.00      0.99    745023
        1.0       0.00      0.00      0.00     18653

avg / total       0.95      0.98      0.96    763676

loss:
0.862909530975


In [14]:
act

array([ 0.02696289,  0.03149135,  0.02563044, ...,  0.02874685,
        0.0132047 ,  0.02696289])

In [13]:
# 保存分类器
from sklearn.externals import joblib
joblib.dump(clf, 'C:\\Users\\work\\Desktop\\tencent\\clf\\0.pkl', compress=3)

['E:\\work\\kaggle\\data\\tecent\\clf\\1.pkl']